In [1]:
!nvidia-smi

Fri Jan  6 01:45:03 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.106.00   Driver Version: 460.106.00   CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-DGXS...  Off  | 00000000:07:00.0 Off |                    0 |
| N/A   51C    P0   298W / 300W |  18066MiB / 32508MiB |     95%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-DGXS...  Off  | 00000000:08:00.0 Off |                    0 |
| N/A   

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="3"

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

2023-01-06 01:45:04.952604: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-06 01:45:05.147178: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7002604964028122203
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 32476168192
locality {
  bus_id: 1
  links {
  }
}
incarnation: 3611980115607987456
physical_device_desc: "device: 0, name: Tesla V100-DGXS-32GB, pci bus id: 0000:0f:00.0, compute capability: 7.0"
xla_global_id: 416903419
]


2023-01-06 01:45:08.526860: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-06 01:45:09.412816: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /device:GPU:0 with 30971 MB memory:  -> device: 0, name: Tesla V100-DGXS-32GB, pci bus id: 0000:0f:00.0, compute capability: 7.0


In [6]:
import gc
import gdown
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from collections import defaultdict
import tqdm.notebook as tqdm

In [7]:
file = 'cellular_component.csv'
if not os.path.exists(file):
    url = "https://drive.google.com/file/d/1eYWHvzPtjrY67-h2iDybuROW0not95dG/view?usp=share_link"
    output = file
    gdown.download(url=url, output=output, quiet=False, fuzzy=True)

In [8]:
df = pd.read_csv(file)
df

,Entry,Entry Name,Organism,Length,Sequence,Gene Ontology (cellular component)
0,F4KFS5,GMI1_ARATH,Arabidopsis thaliana (Mouse-ear cress),1598,MSSRRSVKRSLVLDDDDDEDIFYNFKVLLPNGTSVKLTLKNPEPEI...,GO:0005634
1,Q54NM7,Y6364_DICDI,Dictyostelium discoideum (Slime mold),219,MIINNQNSPQSINTPSSVSSRQHINKSKKKKENVIKRMLIRLSNSN...,GO:0016021
2,B4ESY8,URK_PROMH,Proteus mirabilis (strain HI4320),213,MADTAHQCTIVGIAGASASGKSLIASTLYRELRAQVGDHNIGVIPE...,GO:0005737
3,Q9H479,FN3K_HUMAN,Homo sapiens (Human),309,MEQLLRAELRTATLRAFGGPGAGCISEGRAYDTDAGPVFVKVNRRT...,GO:0005829
4,B4PRE2,DGKH_DROYA,Drosophila yakuba (Fruit fly),1917,MSHLKLDTLHVQRSPRGSRRSSRSSGRSSACSSGSISPVPIIPIIS...,GO:0005737
...,...,...,...,...,...,...
95312,Q8A9V4,ATPB_BACTN,Bacteroides thetaiotaomicron (strain ATCC 2914...,505,MSQIIGHISQVIGPVVDVYFEGTDAELMLPSIHDALEIKRPNGKIL...,GO:0005886;GO:0045261
95313,A1BCE9,MACB3_PARDP,Paracoccus denitrificans (strain Pd 1222),640,MPLIRIRGLHRVFGEGAARAHVLRGIDLDIHAGEFVAIVGTSGSGK...,GO:0016021;GO:0005886
95314,Q9HIS8,RL30_THEAC,Thermoplasma acidophilum (strain ATCC 25905 / ...,165,MLAVIRIRGRTGIKEDIADTAHLMRLNRINHLVLLNENEVVKGMLQ...,GO:0015934
95315,Q3J2I9,AROK_CERS4,Cereibacter sphaeroides (strain ATCC 17023 / D...,199,MKVGAEVRRRGNREDGRQVMARLKKTVVMVGMMGAGKTAVGSALAR...,GO:0005737


In [9]:
go_terms_cc = set()
for idx, row in df.iterrows():
    for term in row['Gene Ontology (cellular component)'].split(';'):
        go_terms_cc.add(term)
go_terms_cc = list(go_terms_cc)
go_terms_cc.sort()
print(len(go_terms_cc))
print(go_terms_cc)

105
['GO:0000015', 'GO:0000139', 'GO:0000428', 'GO:0000776', 'GO:0000785', 'GO:0000786', 'GO:0005576', 'GO:0005615', 'GO:0005634', 'GO:0005635', 'GO:0005654', 'GO:0005667', 'GO:0005694', 'GO:0005730', 'GO:0005737', 'GO:0005739', 'GO:0005741', 'GO:0005743', 'GO:0005758', 'GO:0005759', 'GO:0005764', 'GO:0005765', 'GO:0005768', 'GO:0005769', 'GO:0005773', 'GO:0005774', 'GO:0005777', 'GO:0005783', 'GO:0005789', 'GO:0005794', 'GO:0005802', 'GO:0005813', 'GO:0005829', 'GO:0005833', 'GO:0005840', 'GO:0005856', 'GO:0005874', 'GO:0005886', 'GO:0005887', 'GO:0005925', 'GO:0005929', 'GO:0005938', 'GO:0009279', 'GO:0009295', 'GO:0009317', 'GO:0009376', 'GO:0009380', 'GO:0009507', 'GO:0009523', 'GO:0009535', 'GO:0009536', 'GO:0009570', 'GO:0009897', 'GO:0009986', 'GO:0010008', 'GO:0014069', 'GO:0015629', 'GO:0015934', 'GO:0015935', 'GO:0016020', 'GO:0016021', 'GO:0016282', 'GO:0016323', 'GO:0016324', 'GO:0016607', 'GO:0019013', 'GO:0019031', 'GO:0020002', 'GO:0022625', 'GO:0022627', 'GO:0030176', '

In [10]:
def get_segments(sequence,segment_size=100,gap=30):
    segments = []
    start = 0
    end = segment_size
    while end <= len(sequence):
        segments.append(sequence[start:end])
        start += gap
        end += gap
    last_segment = sequence[start:]
    segments.append(last_segment)
    
    return segments

def get_training_data(df,segment_size=100,gap=30):
    training_data = list()
    for idx,row in tqdm.tqdm(df.iterrows()):
        labels = [0] * len(go_terms_cc)
        for term in row['Gene Ontology (cellular component)'].split(';'):
            labels[go_terms_cc.index(term)] = 1
        segments = get_segments(row['Sequence'],segment_size,gap)
        for segment in segments:
            training_data.append([row['Entry'],segment,labels])
    return training_data

In [11]:
training_data = get_training_data(df)
print(len(training_data))

0it [00:00, ?it/s]

1069179


In [12]:
def get_ngrams(segment,n=3):
    ngrams = []
    for i in range(len(segment)-n+1):
        ngrams.append(segment[i:i+n])
    return ngrams

In [13]:
# Generate training data of ngrams
if os.path.exists('cc/training_data_3grams.npy'):
    training_data_ngrams = np.load('cc/training_data_3grams.npy',allow_pickle=True)
else:
    training_data_ngrams = []

    for i in tqdm.tqdm(range(len(training_data))):
        training_data_ngrams.append([training_data[i][0],get_ngrams(training_data[i][1],n=3),training_data[i][2]])
        
    np.save('cc/training_data_3grams.npy',training_data_ngrams)
    
print(len(training_data_ngrams))

1069179


In [14]:
def get_skip_grams(segment,skip=1,n=3):
    skip_grams = []
    window_size = skip + n
    for i in range(len(segment)-window_size+1):
        window = segment[i:i+window_size]
        indices = list(range(window_size))
        indices.pop(0)
        for idx in indices[::-1]:
            temp = ''
            for j in range(window_size):
                if j!=idx:
                    temp+=window[j]
            skip_grams.append(temp)

    return skip_grams

In [15]:
if os.path.exists('cc/training_data_skip1_3grams.npy'):
    training_data_skip_grams = np.load('cc/training_data_skip1_3grams.npy',allow_pickle=True)
else:
    training_data_skip_grams = []
    for i in tqdm.tqdm(range(len(training_data))):
        training_data_skip_grams.append([training_data[i][0],get_skip_grams(training_data[i][1],n=3),training_data[i][2]])
    np.save('cc/training_data_skip1_3grams.npy',training_data_skip_grams)
print(len(training_data_skip_grams))

1069179


In [16]:
import tensorflow as tf
import tensorflow_addons as tfa

In [17]:
#Assuming training_data as global variable

def train_test_split(X,y,fold_no,prev_index,Kfolds=5):
    test_split = 1/Kfolds
    
    start_index = prev_index
    end_index = (fold_no + 1) * (test_split) * len(X)
    end_index = round(end_index)
    
    if end_index==len(X):
        end_index -= 1
    
    entry = training_data[end_index][0]
    entries = [sample[0] for sample in training_data]
    
    first_occurence = entries.index(entry)
    entries.reverse()
    
    last_occurence = entries.index(entry)
    last_occurence = len(entries) - last_occurence - 1
    
    del entries
    gc.collect()
    
    end_index = first_occurence if (abs(end_index-first_occurence) < abs(end_index-last_occurence)) else last_occurence
    
    X_test = X[start_index:end_index+1]
    y_test = y[start_index:end_index+1]
    X_train = X[:start_index]
    X_train.extend(X[end_index+1:])
    y_train = y[:start_index]
    y_train.extend(y[end_index+1:])
    
    return X_train, y_train, X_test, y_test, start_index, end_index + 1

In [18]:
MAX_WORDS = 13824
MAX_LEN_NG = 98 #100
MAX_LEN_SG = 291 #300

def tokenization(X_train,X_test,maxlen):

    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=MAX_WORDS)
    tokenizer.fit_on_texts(X_train)

    X_train = tokenizer.texts_to_sequences(X_train)
    X_test = tokenizer.texts_to_sequences(X_test)

    vocab_size = len(tokenizer.word_index) + 1

    X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train, padding='post', maxlen=maxlen)
    X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test, padding='post', maxlen=maxlen)
    
    return X_train, X_test, vocab_size, tokenizer

In [19]:
from tensorflow.keras import backend as K

false_negative_penalty = 6
false_positive_penalty = 1

def custom_loss(y_true, y_logit):

    loss = float(0)
    y_true = tf.cast(y_true, tf.float32)
    y_logit = tf.cast(y_logit, tf.float32)
    
    first_term = false_negative_penalty * float(y_true) * - K.log(y_logit + K.epsilon())
    second_term = false_positive_penalty * (1 - float(y_true)) * - K.log(1 - y_logit + K.epsilon())
    
    loss = K.mean(first_term+second_term)

    return loss

def precision(y_true, y_pred):
    
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)), axis=1)
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)), axis=1)
    precision = true_positives / (predicted_positives + K.epsilon())
    
    return K.mean(precision)


def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)), axis=1)
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)), axis=1)
    recall = true_positives / (possible_positives + K.epsilon())
    return K.mean(recall)

def f1_score(y_true, y_pred):
    rec = recall(y_true,y_pred)
    prec = precision(y_true,y_pred)
    f1 = 2*prec*rec/(prec+rec)
    return f1

In [20]:
class attention(tf.keras.layers.Layer):
    
    def __init__(self, return_sequences=True,**kwargs):
        self.return_sequences = return_sequences
        super(attention,self).__init__()

    def get_config(self):
      config = super().get_config().copy()
      config.update({
          'return_sequences': self.return_sequences 
      })
      return config
        
    def build(self, input_shape):
        
        self.W=self.add_weight(name="att_weight", shape=(input_shape[-1],1),
                               initializer="normal")
        self.b=self.add_weight(name="att_bias", shape=(input_shape[1],1),
                               initializer="zeros")
        
        super(attention,self).build(input_shape)
        
    def call(self, x):
        
        e = K.tanh(K.dot(x,self.W)+self.b)
        a = K.softmax(e, axis=1)
        output = x*a
        
        if self.return_sequences:
            return output
        
        return K.sum(output, axis=1)

In [21]:
#Base model using ngrams 
NUM_CLASSES = 105

def get_model_ng(vocab_size_ng):
    input_ngrams = tf.keras.layers.Input(shape=(MAX_LEN_NG,))

    embedding_layer = tf.keras.layers.Embedding(vocab_size_ng, 32)(input_ngrams)

    LSTM_Layer_1 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(70,return_sequences=True))(embedding_layer)
    attention_output_1 = attention(return_sequences=False)(LSTM_Layer_1)
    dropout = tf.keras.layers.Dropout(0.3)(attention_output_1)
    dense_layer_1 = tf.keras.layers.Dense(NUM_CLASSES, activation='sigmoid')(dropout)

    model = tf.keras.models.Model(
        inputs=input_ngrams, 
        outputs=dense_layer_1
    )
    
    return model

In [22]:
#Base model using skip grams 
NUM_CLASSES = 105

def get_model_sg(vocab_size_sg):
    input_skip_grams = tf.keras.layers.Input(shape=(MAX_LEN_SG,))

    embedding_layer = tf.keras.layers.Embedding(vocab_size_sg, 32)(input_skip_grams)

    LSTM_Layer_1 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(70,return_sequences=True))(embedding_layer)
    attention_output_1 = attention(return_sequences=False)(LSTM_Layer_1)
    dropout = tf.keras.layers.Dropout(0.3)(attention_output_1)
    dense_layer_1 = tf.keras.layers.Dense(NUM_CLASSES, activation='sigmoid')(dropout)

    model = tf.keras.models.Model(
        inputs=input_skip_grams, 
        outputs=dense_layer_1
    )
    
    return model

In [23]:
NUM_CLASSES = 105 #For molecular function (Change according to aspects)

def get_model_ng_sg(vocab_size_ng, vocab_size_sg):
    #Input layers

    input_ngrams = tf.keras.layers.Input(shape=(MAX_LEN_NG,)) 
    input_skip_grams = tf.keras.layers.Input(shape=(MAX_LEN_SG,)) 

    #embeddings
    embedding_layer_ngrams = tf.keras.layers.Embedding(vocab_size_ng, 32)(input_ngrams)
    embedding_layer_skip_grams = tf.keras.layers.Embedding(vocab_size_sg, 32)(input_skip_grams)

    #BI-LSTMs for each of the inputs
    sequence_output_1 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(70,return_sequences=True))(embedding_layer_ngrams)
    attention_output_1 = attention(return_sequences=False)(sequence_output_1)
    dropout_1 = tf.keras.layers.Dropout(0.3)(attention_output_1)
    dense_layer_1 = tf.keras.layers.Dense(NUM_CLASSES, activation='sigmoid')(dropout_1)

    sequence_output_2 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(70, return_sequences=True))(embedding_layer_skip_grams)
    attention_output_2 = attention(return_sequences=False)(sequence_output_2)
    dropout_2 = tf.keras.layers.Dropout(0.3)(attention_output_2)
    dense_layer_2 = tf.keras.layers.Dense(NUM_CLASSES, activation='sigmoid')(dropout_2)

    max_layer = tf.keras.layers.Maximum()([dense_layer_1,dense_layer_2])

    model = tf.keras.models.Model(
        inputs=[
            input_ngrams,
            input_skip_grams
        ], 
        outputs=max_layer)
    
    return model

In [24]:
def compute_metrics(predictions, start_index, end_index):
    final_predictions = []
    actual_y_test = []

    current_entry = ''
    counter = 0
    total_counts = 0

    if len(predictions) == len(training_data[start_index: end_index]):
        temp = np.zeros(NUM_CLASSES)
        for i in range(len(predictions)):
            if current_entry != training_data[start_index+i][0]:
                #compute prev
                if i!=0:
                    temp /= counter
                    final_predictions.append(temp)

                #reset
                total_counts += counter
                counter = 1
                temp = np.zeros(NUM_CLASSES)

                #init new
                current_entry = training_data[start_index+i][0]
                temp += np.array(predictions[i])
                actual_y_test.append(training_data[start_index+i][2])
            else:
                temp += np.array(predictions[i])
                counter += 1

        total_counts += counter
        temp /= counter
        final_predictions.append(temp)

    else:
        print('Lengths of predictions dont match with test data')
    
    final_predictions = np.array(final_predictions, dtype=float)
    actual_y_test = np.array(actual_y_test, dtype=float)
    
    rec = recall(actual_y_test,final_predictions)
    prec = precision(actual_y_test,final_predictions)
    f1 = f1_score(actual_y_test,final_predictions)
    
    return rec,prec,f1

### Model using ngrams only

In [37]:
#Considering ngrams
X_ng = [' '.join(sample[1]) for sample in training_data_ngrams]
y = [sample[2] for sample in training_data]

del training_data_ngrams

In [39]:
Kfolds = 5
prev_index = 0

recs = []
precs = []
f1s = []

for i in range(Kfolds):
    print('\n\n****Fold:', i+1,'****')
    
    print('Splitting into train-test...')
    X_train_ng, y_train, X_test_ng, y_test, start_index, prev_index = train_test_split(X_ng,y,i,prev_index,Kfolds)

    print('Tokenizing...')
    X_train_ng, X_test_ng, vocab_size_ng, tokenizer1 = tokenization(X_train_ng, X_test_ng, MAX_LEN_NG)
    
    print('Shuffling...')
    shuffled = [[X_train_ng[i],y_train[i]] for i in range(len(X_train_ng))]
    np.random.shuffle(shuffled)

    X_train_ng = [shuffled[i][0] for i in range(len(shuffled))]
    y_train = [shuffled[i][1] for i in range(len(shuffled))]
    X_train_ng = np.array(X_train_ng)
    y_train = np.array(y_train)
    
    
    
    model = get_model_ng(vocab_size_ng)
    
    model.compile(
        loss=custom_loss, 
        optimizer='adam', 
        metrics=[
            recall,
            precision,
            f1_score
        ])
    
    print('Training...')
    history = model.fit(X_train_ng, y_train, batch_size=32, epochs=5,validation_split=0.2)
    
    print('\nEvaluating model...')
    predictions = model.predict(X_test_ng)
    
    rec, prec, f1 = compute_metrics(predictions, start_index, prev_index)
    
    recs.append(rec.numpy())
    precs.append(prec.numpy())
    f1s.append(f1.numpy())

print('Recall:',sum(recs)/len(recs))
print('Precision:',sum(precs)/len(precs))
print('F1-Score:',sum(f1s)/len(f1s))



****Fold: 1 ****
Splitting into train-test...
Tokenizing...
Shuffling...


2023-01-05 15:21:42.615791: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30971 MB memory:  -> device: 0, name: Tesla V100-DGXS-32GB, pci bus id: 0000:0f:00.0, compute capability: 7.0


Training...
Epoch 1/5


2023-01-05 15:21:52.100608: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100


21384/21384 [==============================] - 263s 12ms/step - loss: 0.1983 - recall: 0.5760 - precision: 0.4222 - f1_score: 0.4845 - val_loss: 0.1778 - val_recall: 0.6283 - val_precision: 0.4645 - val_f1_score: 0.5328
Epoch 2/5
21384/21384 [==============================] - 252s 12ms/step - loss: 0.1716 - recall: 0.6421 - precision: 0.4806 - f1_score: 0.5483 - val_loss: 0.1645 - val_recall: 0.6627 - val_precision: 0.4926 - val_f1_score: 0.5639
Epoch 3/5
21384/21384 [==============================] - 252s 12ms/step - loss: 0.1594 - recall: 0.6663 - precision: 0.5059 - f1_score: 0.5738 - val_loss: 0.1565 - val_recall: 0.6611 - val_precision: 0.5324 - val_f1_score: 0.5886
Epoch 4/5
21384/21384 [==============================] - 253s 12ms/step - loss: 0.1505 - recall: 0.6837 - precision: 0.5212 - f1_score: 0.5902 - val_loss: 0.1503 - val_recall: 0.6809 - val_precision: 0.5342 - val_f1_score: 0.5976
Epoch 5/5
21384/21384 [==============================] - 252s 12ms/step - loss: 0.1434 - r

### Model using skip grams only

In [26]:
#Considering skip grams
X_sg = [' '.join(sample[1]) for sample in training_data_skip_grams]
y = [sample[2] for sample in training_data]

del training_data_skip_grams

In [ ]:
Kfolds = 5
prev_index = 0

recs = []
precs = []
f1s = []

for i in range(Kfolds):
    print('\n\n****Fold:', i+1,'****')
    
    print('Splitting into train-test...')
    X_train_sg, y_train, X_test_sg, y_test, start_index, prev_index = train_test_split(X_sg,y,i,prev_index,Kfolds)

    print('Tokenizing...')
    X_train_sg, X_test_sg, vocab_size_sg, tokenizer1 = tokenization(X_train_sg, X_test_sg, MAX_LEN_SG)
    
    print('Shuffling...')
    shuffled = [[X_train_sg[i],y_train[i]] for i in range(len(X_train_sg))]
    np.random.shuffle(shuffled)

    X_train_sg = [shuffled[i][0] for i in range(len(shuffled))]
    y_train = [shuffled[i][1] for i in range(len(shuffled))]
    X_train_sg = np.array(X_train_sg)
    y_train = np.array(y_train)
    

    model = get_model_sg(vocab_size_sg)
    
    model.compile(
        loss=custom_loss, 
        optimizer='adam', 
        metrics=[
            recall,
            precision,
            f1_score
        ])
    
    print('Training...')
    history = model.fit(X_train_sg, y_train, batch_size=32, epochs=5,validation_split=0.2)
    
    print('Evaluating model...')
    predictions = model.predict(X_test_sg)
    
    rec, prec, f1 = compute_metrics(predictions, start_index, prev_index)
    
    recs.append(rec.numpy())
    precs.append(prec.numpy())
    f1s.append(f1.numpy())

print('Recall:',sum(recs)/len(recs))
print('Precision:',sum(precs)/len(precs))
print('F1-Score:',sum(f1s)/len(f1s))



****Fold: 1 ****
Splitting into train-test...
Tokenizing...
Shuffling...


2023-01-05 17:39:47.284624: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30971 MB memory:  -> device: 0, name: Tesla V100-DGXS-32GB, pci bus id: 0000:0f:00.0, compute capability: 7.0


Training...
Epoch 1/5


2023-01-05 17:39:53.993803: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100


21384/21384 [==============================] - 557s 26ms/step - loss: 0.1993 - recall: 0.5741 - precision: 0.4177 - f1_score: 0.4810 - val_loss: 0.1806 - val_recall: 0.6024 - val_precision: 0.4882 - val_f1_score: 0.5380
Epoch 2/5
21384/21384 [==============================] - 551s 26ms/step - loss: 0.1734 - recall: 0.6401 - precision: 0.4770 - f1_score: 0.5452 - val_loss: 0.1662 - val_recall: 0.6701 - val_precision: 0.4749 - val_f1_score: 0.5545
Epoch 3/5
21384/21384 [==============================] - 549s 26ms/step - loss: 0.1606 - recall: 0.6654 - precision: 0.5028 - f1_score: 0.5714 - val_loss: 0.1579 - val_recall: 0.6658 - val_precision: 0.5175 - val_f1_score: 0.5812
Epoch 4/5
21384/21384 [==============================] - 552s 26ms/step - loss: 0.1517 - recall: 0.6815 - precision: 0.5180 - f1_score: 0.5873 - val_loss: 0.1516 - val_recall: 0.6782 - val_precision: 0.5285 - val_f1_score: 0.5929
Epoch 5/5
21384/21384 [==============================] - 551s 26ms/step - loss: 0.1449 - r

### Model using skip grams and ngrams

In [25]:
#Considering ngrams and skip grams
X_ng = [' '.join(sample[1]) for sample in training_data_ngrams]
X_sg = [' '.join(sample[1]) for sample in training_data_skip_grams]
y = [sample[2] for sample in training_data]

del training_data_ngrams
del training_data_skip_grams

In [26]:
print(len(X_ng),len(X_sg))

1069179 1069179


In [27]:
Kfolds = 5
prev_index = 0

recs = []
precs = []
f1s = []

for i in range(Kfolds):
    print('\n\n****Fold:', i+1,'****')
    
    print('Splitting into train-test...')
    X_train_ng, y_train, X_test_ng, y_test, start_index, prev_index1 = train_test_split(X_ng,y,i,prev_index,Kfolds)
    X_train_sg, _, X_test_sg, _, _, _= train_test_split(X_sg,y,i,prev_index,Kfolds)
    
    prev_index = prev_index1
    
    print('Tokenizing...')
    X_train_ng, X_test_ng, vocab_size_ng, tokenizer1 = tokenization(X_train_ng, X_test_ng, MAX_LEN_NG)  
    X_train_sg, X_test_sg, vocab_size_sg, tokenizer2 = tokenization(X_train_sg, X_test_sg, MAX_LEN_SG)
    
   
    print('Shuffling...')   
    shuffled = [[X_train_ng[i],X_train_sg[i],y_train[i]] for i in range(len(X_train_ng))]
    np.random.shuffle(shuffled)

    X_train_ng = [shuffled[i][0] for i in range(len(shuffled))]
    X_train_sg = [shuffled[i][1] for i in range(len(shuffled))]
    y_train = [shuffled[i][2] for i in range(len(shuffled))]
    X_train_ng = np.array(X_train_ng)
    X_train_sg = np.array(X_train_sg)
    y_train = np.array(y_train)
    
    
    model = get_model_ng_sg(vocab_size_ng, vocab_size_sg)
    
    model.compile(
        loss=custom_loss, 
        optimizer='adam', 
        metrics=[
            recall,
            precision,
            f1_score
        ])

    print('Training...')
    history = model.fit([X_train_ng,X_train_sg], y_train, batch_size=32, epochs=5,validation_split=0.2)
    
    print('Evaluating model...')
    predictions = model.predict([X_test_ng,X_test_sg])
    
    rec, prec, f1 = compute_metrics(predictions, start_index, prev_index)
    
    recs.append(rec.numpy())
    precs.append(prec.numpy())
    f1s.append(f1.numpy())

print('Recall:',sum(recs)/len(recs))
print('Precision:',sum(precs)/len(precs))
print('F1-Score:',sum(f1s)/len(f1s))



****Fold: 1 ****
Splitting into train-test...
Tokenizing...
Shuffling...


2023-01-06 01:54:29.048860: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30971 MB memory:  -> device: 0, name: Tesla V100-DGXS-32GB, pci bus id: 0000:0f:00.0, compute capability: 7.0


Training...
Epoch 1/5


2023-01-06 01:54:39.482304: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100


21384/21384 [==============================] - 719s 33ms/step - loss: 0.1974 - recall: 0.5788 - precision: 0.4226 - f1_score: 0.4859 - val_loss: 0.1784 - val_recall: 0.6294 - val_precision: 0.4571 - val_f1_score: 0.5282
Epoch 2/5
21384/21384 [==============================] - 710s 33ms/step - loss: 0.1744 - recall: 0.6373 - precision: 0.4743 - f1_score: 0.5423 - val_loss: 0.1682 - val_recall: 0.6308 - val_precision: 0.5170 - val_f1_score: 0.5672
Epoch 3/5
21384/21384 [==============================] - 711s 33ms/step - loss: 0.1636 - recall: 0.6590 - precision: 0.4983 - f1_score: 0.5661 - val_loss: 0.1599 - val_recall: 0.6737 - val_precision: 0.5002 - val_f1_score: 0.5729
Epoch 4/5
21384/21384 [==============================] - 710s 33ms/step - loss: 0.1551 - recall: 0.6740 - precision: 0.5142 - f1_score: 0.5820 - val_loss: 0.1546 - val_recall: 0.6806 - val_precision: 0.5139 - val_f1_score: 0.5844
Epoch 5/5
21384/21384 [==============================] - 709s 33ms/step - loss: 0.1485 - r